In [1]:
import torch
from transformers import ViTForImageClassification, ViTFeatureExtractor
from torch import nn


2025-02-24 11:39:59.253900: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-24 11:40:08.503408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-24 11:40:11.722625: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-24 11:40:12.848203: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-24 11:40:19.802806: I tensorflow/core/platform/cpu_feature_guar

In [67]:
# Load the pre-trained Vision Transformer model
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k")

# Load the corresponding feature extractor (used to preprocess input images)
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/macula/SMATousi/.conda/envs/snorkel/lib/python3.12/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [41]:
# Print the full architecture of the model
print(model)
# print(feature_extractor)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [24]:
feature_extractor.do_resize = False
feature_extractor.do_rescale = False

In [68]:
from PIL import Image
import requests

image = Image.open("/home1/pupil/SMATousi/Pos_Neg_224/pos/rgb_images/rgb_5_tile_4716.tif")
# image = Image.open("/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_pos/rgb_images/rgb_5_tile_4516.tif")
# image.show()

# Preprocess the image
inputs = feature_extractor(images=image, return_tensors="pt")
embedding_layer = model.vit.embeddings

# torch.nn.init.normal_(embedding_layer.patch_embeddings.projection.weight, mean=0.0, std=0.02)


# Make predictions
with torch.no_grad():
    embedding_output = embedding_layer(**inputs)
    print("Embedding Output Shape: ", embedding_output.shape)
    
    encoder_output = model.vit.encoder(embedding_output)
    print("Encoder Output Shape: ", encoder_output[0].shape)
    
    layer_norm_output = model.vit.layernorm(encoder_output[0])
    print("Layer Norm Output Shape: ", layer_norm_output.shape)

    # Pooling: take the first token (CLS token) from the sequence
    pooled_output = layer_norm_output[:, 0]
    print("Pooled Output Shape: ", pooled_output.shape)
    
    logits = model.classifier(pooled_output)
    print("Logits Output Shape: ", logits.shape)

    outputs = model(**inputs)
    print("Full Model Output Shape: ", outputs[0].shape)

# Get logits
# logits = outputs.logits
# print(logits)

Embedding Output Shape:  torch.Size([1, 197, 768])
Encoder Output Shape:  torch.Size([1, 197, 768])
Layer Norm Output Shape:  torch.Size([1, 197, 768])
Pooled Output Shape:  torch.Size([1, 768])
Logits Output Shape:  torch.Size([1, 2])
Full Model Output Shape:  torch.Size([1, 2])


In [72]:
for parameter in model.vit.encoder.parameters():
    parameter.require_grad = False

In [70]:
outputs

ImageClassifierOutput(loss=None, logits=tensor([[-0.0420, -0.0846]]), hidden_states=None, attentions=None)